<a href="https://colab.research.google.com/github/ewotawa/secure_private_ai/blob/master/Section_2_Federated_Learning_Final_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Federated Learning Final Project

## Overview
* See  <a href="https://classroom.udacity.com/nanodegrees/nd185/parts/3fe1bb10-68d7-4d84-9c99-9539dedffad5/modules/28d685f0-0cb1-4f94-a8ea-2e16614ab421/lessons/c8fe481d-81ea-41be-8206-06d2deeb8575/concepts/a5fb4b4c-e38a-48de-b2a7-4e853c62acbe">video</a> for additional details. 
* Do Federated Learning where the central server is not trusted with the raw gradients.  
* In the final project notebook, you'll receive a dataset.  
* Train on the dataset using Federated Learning.  
* The gradients should not come up to the server in raw form.  
* Instead, use the new .move() command to move all of the gradients to one of the workers, sum them up there, and then bring that batch up to the central server and then bring that batch up 
* Idea: the central server never actually sees the raw gradient for any person.  
* We'll look at secure aggregation in course 3.  
* For now, do a larger-scale Federated Learning case where you handle the gradients in a special way.

## Approach
* Reviewing methods of classmates for Federated Learning. 

## References
*  <a href = "https://github.com/edgarinvillegas/private-ai/blob/master/Section%203%20-%20Final%20project.ipynb/">GitHub Notebook</a>


### Install libraries and dependencies

In [1]:
# PySyft

!pip install syft

import syft as sy

# PyTorch

!pip install torch
!pip install torchvision

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import datasets, transforms

# Numpy

import numpy as np

     |████████████████████████████████| 225kB 42.9MB/s 
     |████████████████████████████████| 389kB 48.2MB/s 
     |████████████████████████████████| 256kB 61.1MB/s 
     |████████████████████████████████| 460kB 50.6MB/s 
     |████████████████████████████████| 204kB 49.7MB/s 
     |████████████████████████████████| 2.1MB 47.2MB/s 
     |████████████████████████████████| 81kB 33.4MB/s 
     |████████████████████████████████| 276kB 59.4MB/s 
     |████████████████████████████████| 51kB 25.4MB/s 
     |████████████████████████████████| 122kB 27.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
      Successfully uninstalled msgpack-0.5.6
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
  

W0721 06:17:45.494817 139874483152768 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0721 06:17:45.508292 139874483152768 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



###  Recall Toy Federated Learning

Use the data and model from Section 2

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [0]:
# A Toy Dataset
data = torch.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = torch.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# A Toy Model
model = nn.Linear(2,1)

In [0]:
# Optimizer
opt = optim.SGD(params=model.parameters(), lr=0.1)

### Federated Learning

Set up hook, virtual workers, and virtual aggregator

In [106]:
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
vw00 = sy.VirtualWorker(hook, id="vw00")
vw01 = sy.VirtualWorker(hook, id="vw01")

aggr = sy.VirtualWorker(hook, id="aggr")

W0721 07:50:33.475393 139874483152768 hook.py:98] Torch was already hooked... skipping hooking process


In [107]:
vw00.clear_objects()
vw01.clear_objects()
aggr.clear_objects()

<VirtualWorker id:aggr #objects:0>

In [108]:
data_vw00 = data[0:2].send(vw00)
target_vw00 = target[0:2].send(vw00)

vw00._objects

{11294118747: tensor([[1., 1.],
         [0., 1.]], requires_grad=True), 70488089605: tensor([[1.],
         [1.]], requires_grad=True)}

In [109]:
data_vw01 = data[2:4].send(vw01)
target_vw01 = target[2:4].send(vw01)

vw01._objects

{3132272333: tensor([[1., 0.],
         [0., 0.]], requires_grad=True), 85255226377: tensor([[0.],
         [0.]], requires_grad=True)}

In [0]:
datasets = [(data_vw00, target_vw00), (data_vw01, target_vw01)]

In [0]:
def fed_train(iterations=20):

    # Need to define separate model and optimizer for each worker. Need to move the models to the workers

    models = {
        'vw00': nn.Linear(2,1).send('vw00'),
        'vw01': nn.Linear(2,1).send('vw01')
    }

    opts = {
        'vw00': optim.SGD(params=models['vw00'].parameters(), lr=0.1),
        'vw01': optim.SGD(params=models['vw01'].parameters(), lr=0.1)
    }
    
    for iter in range(iterations):
        
        print(iter)

        for _data, _target in datasets:

            # locate the data, identify model, optimizer by dataset ids
            worker_id = _data.location.id
            model = models[worker_id]
            opt = opts[worker_id]
            
            print("data location: ", _data.location, "\tworker ID: ", worker_id)
            
            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

    return models, model.parameters()
            

In [112]:
models, params = fed_train()

0
data location:  <VirtualWorker id:vw00 #objects:4> 	worker ID:  vw00
data location:  <VirtualWorker id:vw01 #objects:4> 	worker ID:  vw01
1
data location:  <VirtualWorker id:vw00 #objects:4> 	worker ID:  vw00
data location:  <VirtualWorker id:vw01 #objects:4> 	worker ID:  vw01
2
data location:  <VirtualWorker id:vw00 #objects:4> 	worker ID:  vw00
data location:  <VirtualWorker id:vw01 #objects:4> 	worker ID:  vw01
3
data location:  <VirtualWorker id:vw00 #objects:4> 	worker ID:  vw00
data location:  <VirtualWorker id:vw01 #objects:4> 	worker ID:  vw01
4
data location:  <VirtualWorker id:vw00 #objects:4> 	worker ID:  vw00
data location:  <VirtualWorker id:vw01 #objects:4> 	worker ID:  vw01
5
data location:  <VirtualWorker id:vw00 #objects:4> 	worker ID:  vw00
data location:  <VirtualWorker id:vw01 #objects:4> 	worker ID:  vw01
6
data location:  <VirtualWorker id:vw00 #objects:4> 	worker ID:  vw00
data location:  <VirtualWorker id:vw01 #objects:4> 	worker ID:  vw01
7
data location:  <V

In [113]:
aggr._objects

{}

In [114]:
print(models)

{'vw00': Linear(in_features=2, out_features=1, bias=True), 'vw01': Linear(in_features=2, out_features=1, bias=True)}
